<h1>Fit a CNN on the pngs files</h1>

In [ ]:
# !pip install tensorflow

In [2]:
"""
    Fit a CNN per machine (on the pngs files in machine/png_train)
    from: https://keras.io/examples/vision/image_classification_from_scratch/
"""
import os 
import datetime
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# add parent folder to path 
from pathlib import Path
path = Path(os.getcwd())
str_parent = str(path.parent.absolute())
if not str_parent in sys.path:
    sys.path.append(str_parent)
import utils 
from SoundFile import SoundFile



In [3]:
image_size = (333, 216)
batch_size = 32
epochs = 8
data_augmentation = keras.Sequential(
    [
        # layers.experimental.preprocessing.RandomFlip("horizontal"),
        # layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)


In [6]:

# Initialising the CNN v4: make model
def make_model(input_shape):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)    
    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)


In [7]:

model = make_model(input_shape=image_size + (3,))
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)


In [8]:

# Part 2 - Fitting the CNN to the images
for folder_machine in utils.list_datasets: 
    use_folder = '../../_data_png/' + folder_machine + '/png_train'
    print('Processing: ', use_folder, '...')
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        use_folder,
        validation_split=0.2,
        subset="training",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        use_folder,
        validation_split=0.2,
        subset="validation",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )
    model.fit(
        train_ds, epochs=epochs, validation_data=val_ds,
    )

    # save the model to disk for this machine ----------------------- ' + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S") + '_
    filename_classifier = '../../_classifiers/cnn_' + folder_machine + '.h5'
    model.save(filename_classifier)
    print(folder_machine, ': model saved: ', filename_classifier)


Processing:  ../../_data_png/valve/png_train ...
Found 3678 files belonging to 2 classes.
Using 2943 files for training.
Found 3678 files belonging to 2 classes.
Using 735 files for validation.
Epoch 1/8
 1/92 [..............................] - ETA: 11:11 - loss: 0.7288 - accuracy: 0.4375

KeyboardInterrupt: 